In [61]:
# Description of steps:
# 1. Import existing dataset of indiegogo and create a new dataset which will be used
#    to store the scraped information.
# 2. By assigning each unique project URL to the chromedriver, beautiful soup scraps
#    the information we want. The class which is identified was found by inspecting
#    a random indiegogo project webpage - CTRL + SHIFT + I in Chrome. 
# 3. Iterate the above process for all projects available by using the project ID
# 4. Create updated dataset by adding the new information extracted to the old dataset.
# 5. Export excel file with the new information.

# I used anaconda prompt for python to download all packages first. 
# Language: Python 3.6.

# Import all libraries.
from selenium import webdriver
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import xlsxwriter

# Import the dataset - it needs to be on the same folder as the python document.
old_indiegogo = pd.read_excel('Nassia1.xlsx')

# Create new dataset which will be used to include the scraped information.
new_indiegogo = old_indiegogo.filter(['ProjectID','Project_Title'], axis=1)
new_indiegogo.columns = ['ProjectID','Project_Story']
new_indiegogo['ProjectID'] = new_indiegogo.ProjectID.astype(str)

In [57]:
# Iterate through each row of our new data set and by indexing
# the project ID, scrap the respective webpage.

for index, row in new_indiegogo.iterrows():
    
    # The ID needs to be a string in order to be identified as part of
    # the URL.
    project_id = str(row['ProjectID'])
    
    url = 'https://www.indiegogo.com/projects/' + project_id
    
    # Open driver by adding the path where its located on the computer.
    driverPath = 'C:/Users/nassi/Desktop/chromedriver.exe'
    
    # Create the options of the chrome driver which will be used in
    # a later stage for better use.
    options = webdriver.ChromeOptions()
    
    # When scraping each webpage do not open the browser.
    options.add_argument("headless")
    
    # Use the above information to define the driver which will scrap the data.
    driver = webdriver.Chrome(executable_path=driverPath, chrome_options=options)
    
    # Wait only 4 seconds.
    driver.implicitly_wait(4)
    
    # Run each unique URL with the driver.
    driver.get(url)
    
    # Create beautiful Soup object for scraping.
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    # Create object which includes only the information under the 
    # class: routerContentStory-storyBody. 
    # Convert to string in order to perform slicing.
    content = str(soup.find_all('div', {'class': 'routerContentStory-storyBody'}))
    
    # Remove all HTML/CSS commands.
    while content.find('<')!=-1 and content.find('>')!=-1:
        content = content[:content.find('<')] + content[content.find('>') + 1:]
    
    # Further "prettify" by removing unnecessary characters.
    content_new = ''.join(content)
    content_new = content_new.replace(', ' , '')
    content_new = content_new.replace('[' , '')
    content_new = content_new.replace(']' , '')
    content_new = content_new.replace('  ' , '')
    content_new = content_new.replace('\n' , ' ')
    
    # Add content to each unique projects 'Project_Story'.
    row['Project_Story'] = content_new

C:\Users\nassi\Anaconda3\envs\python36\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: use options instead of chrome_options


In [58]:
# Add the new column information to the old dataset.
old_indiegogo['Project_Story'] = new_indiegogo['Project_Story']

In [53]:
# Create final_indiegogo.xlsx document in which the final data for all projects will be added.
writer = pd.ExcelWriter('final_indiegogo.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
old_indiegogo.to_excel(writer)

# Output the Excel file.
writer.save()